In [2]:
%matplotlib inline

In [3]:
import pandas as pd
from enum import Enum
from interface import implements, Interface
import random

In [4]:
df = pd.read_csv("train.csv")

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 11 columns):
index_0       9 non-null object
index_1       9 non-null object
index_2       9 non-null object
index_3       9 non-null object
index_4       9 non-null object
index_5       9 non-null object
index_6       9 non-null object
index_7       9 non-null object
Index_8       9 non-null object
whose_turn    9 non-null object
move          9 non-null int64
dtypes: int64(1), object(10)
memory usage: 872.0+ bytes


In [6]:
df.head()

,index_0,index_1,index_2,index_3,index_4,index_5,index_6,index_7,Index_8,whose_turn,move
0,_,_,_,_,_,_,_,_,_,x,0
1,x,_,_,_,_,_,_,_,_,o,4
2,x,_,_,_,o,_,_,_,_,x,6
3,x,_,_,_,o,_,x,_,_,o,3
4,x,_,_,o,o,_,x,_,_,x,5


In [7]:
class Space(Enum):
    """Represents a single space by a player or an empty spot

    """
    def __str__(self):
        if self.value == 1:
            return '_'
        elif self.value == 2:
            return 'x'
        elif self.value == 3:
            return 'o'
        else:
            return '?'

    EMPTY = 1
    X = 2
    O = 3

In [8]:
class Move():
    """Represents a single attempted move by a player
    
    """
    def __init__(self, space: Space, index: int):
            self.space = space
            self.index = index
            
    def __repr__(self):
        return f"Player {self.space} attempted to move at index {self.index}"

In [9]:
class PlayerStrategy(Interface):
    """Defines a purely random strategy
    
    """
    def __init__(self, space: Space):
        pass

    def query_move(self, board: "Board") -> Move:
        pass

In [10]:
class RandomPlayerStrategy(implements(PlayerStrategy)):
    """Defines a purely random strategy
    
    """
    def __init__(self, space: Space):
        self.space = space

    def query_move(self, board: "Board") -> Move:
        return Move(self.space, random.randint(0, 8)) # hardcoded 9 here, its fine

In [27]:
class HumanPlayerStrategy(implements(PlayerStrategy)):
    """Defines a purely random strategy
    
    """
    def __init__(self, space: Space):
        self.space = space

    def query_move(self, board: "Board") -> Move:
        print("Your move, current board:")
        print(str(type(board)))
        print(board)
        print(f"Place a {self.space} at index[0-8]:")
        valid_input = False
        
        while not valid_input:
            try:
                user_input = input()
                index = int(user_input)
                if index < 0 or index > 8:
                    print(f"You entered an invalid index {user_input}, please enter one between [0-8]")
                else:
                    valid_input = True
            except:
                print(f"You entered an invalid index {user_input}, please enter one between [0-8]")
              
        print(f"you entered {index}")
              
        return Move(self.space, index)

In [32]:
class Board:
    """Represents as tic tac toe board
    0 1 2
    3 4 5
    6 7 8
    
    """
    
    class MoveAlreadyTakenException(Exception):
        """Represents an exception when an already made move was attempted
        again.
        
        """
        pass

    def __init__(self, player_x: PlayerStrategy, player_o: PlayerStrategy):
        self.player_x = player_x
        self.player_o = player_o
        self.board = [
            Space.EMPTY, Space.EMPTY, Space.EMPTY,
            Space.EMPTY, Space.EMPTY, Space.EMPTY,
            Space.EMPTY, Space.EMPTY, Space.EMPTY
        ]
    
    def _attempt_move(self, move: Move):
        """Attemts to perform a single move
        
        """
        
        if self.board[move.index] != Space.EMPTY:
            raise Board.MoveAlreadyTakenException()
            
        self.board[move.index] = move.space
            
    def _get_winner(self):
        """Determining who has won. 
        Bad code ahead!
        
        """
        def _set_contain_winner(set_to_check):
            """Checks if this set (horizontal, vertical or diagnol) is a winner
            
            """
            if Space.EMPTY not in set_to_check and len(set_to_check) == 1:
                return True
            return False
        
        horizontals = []
        verticals = []
        diagnols = []
        
        horizontals.append(set([self.board[0], self.board[1], self.board[2]]))            
        horizontals.append(set([self.board[3], self.board[4], self.board[5]]))            
        horizontals.append(set([self.board[6], self.board[7], self.board[8]]))         
        
        verticals.append(set([self.board[0], self.board[3], self.board[6]]))            
        verticals.append(set([self.board[1], self.board[5], self.board[7]]))            
        verticals.append(set([self.board[2], self.board[5], self.board[8]]))   
        
        diagnols.append(set([self.board[0], self.board[4], self.board[8]]))            
        diagnols.append(set([self.board[2], self.board[4], self.board[6]]))  
        
        for horizontal in horizontals:
            if(_set_contain_winner(horizontal)):
                return horizontal.pop() # just to get to element, set is useless now anyways
            
        for vertical in verticals:
            if(_set_contain_winner(vertical)):
                return vertical.pop() # just to get to element, set is useless now anyways
            
        for diagnol in diagnols:
            if(_set_contain_winner(diagnol)):
                return diagnol.pop() # just to get to element, set is useless now anyways
        
    def _has_at_least_one_empty_space(self):
        for i in range(len(self.board)):
            if self.board[i] == Space.EMPTY:
                return True
        return False
        
    def start(self):
        """Starts the game
        
        """
        is_player_x_turn = True
        while (self._get_winner() == None) and (self._has_at_least_one_empty_space()):
            if is_player_x_turn:
                attempted_move = self.player_x.query_move(self)
            else:
                attempted_move = self.player_o.query_move(self)

            print(attempted_move)

            try:
                self._attempt_move(attempted_move)
                is_player_x_turn = not is_player_x_turn
            except Board.MoveAlreadyTakenException:
                print(f"\tmove failed, already taken")
        
        print(self)
        print(f"{self._get_winner()} has won!")
            
    def __str__(self):
        output = ""
        
        for i in range(len(self.board)):
            output += str(self.board[i])
            output += ' '
            
            # prints new lines at end of rows
            if (i!= 0) and (((i + 1) % 3) == 0):
                output += '\n'
        
        return output

In [33]:
print(str(type(board)))
print(board)

<class '__main__.Board'>
_ _ _ 
_ _ _ 
_ _ _ 



In [34]:
board = Board(HumanPlayerStrategy(Space.X), RandomPlayerStrategy(Space.O))
board.start()

Your move, current board:
<class '__main__.Board'>
_ _ _ 
_ _ _ 
_ _ _ 

Place a x at index[0-8]:


 0


you entered 0
Player x attempted to move at index 0
Player o attempted to move at index 8
Your move, current board:
<class '__main__.Board'>
x _ _ 
_ _ _ 
_ _ o 

Place a x at index[0-8]:


 1


you entered 1
Player x attempted to move at index 1
Player o attempted to move at index 4
Your move, current board:
<class '__main__.Board'>
x x _ 
_ o _ 
_ _ o 

Place a x at index[0-8]:


 2


you entered 2
Player x attempted to move at index 2
x x x 
_ o _ 
_ _ o 

x has won!
